In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
trends = pd.read_csv('merge_data/trends_clean.csv')
trends = trends.dropna(subset = ['merge_data/county_fips'])
evapotranspiration = pd.read_csv('merge_data/et_cleaned.csv')

#Climate dataset without cooling and heating days included. Use 'climate_all_data_clean.csv' to include them.
climate = pd.read_csv('merge_data/climate_all_data_clean_without_heat-cool-days.csv')

#Water use dataset without potentially redundant 'totals' columns. Use 'water-use_cleaned.csv' to include them.
water_use = pd.read_csv('merge_data/water-use_cleaned_totals-removed.csv')

In [3]:
water_use['FIPS'] = water_use['FIPS'].astype(str)
trends['county_fips'] = trends['county_fips'].astype(int)
trends['county_fips'] = trends['county_fips'].astype(str)
climate['county_fips'] = climate['county_fips'].astype(int)
climate['county_fips'] = climate['county_fips'].astype(str)

In [4]:
#if CSV files are causing issues with keeping the fips codes as strings, this code will need to run to add leading 0
for i, row in trends.iterrows():
    fips = row['county_fips']
    if len(fips)== 4:
        trends.loc[i, 'county_fips'] = '0' + fips

for i, row in water_use.iterrows():
    fips = row['FIPS']
    if len(fips)== 4:
        water_use.loc[i, 'FIPS'] = '0' + fips
        
for i, row in climate.iterrows():
    fips = row['county_fips']
    if len(fips)== 4:
        climate.loc[i, 'county_fips'] = '0' + fips

Now, we merge all of the datasets

In [5]:
trends_select = trends[['site_id','slope', 'decadal_change', 'aquifer_type_id', 'well_depth', 'county_fips']]

In [6]:
water_use.rename(columns={'FIPS': 'county_fips'}, inplace = True)
merged_df = trends_select.merge(water_use, how = 'left', on = 'county_fips')
merged_df = merged_df.merge(climate, how = 'left', on = 'county_fips')
merged_df = merged_df.merge(evapotranspiration, how = 'left', on = 'site_id')

In [7]:
#replace missing well depth values with mean
rep = merged_df['well_depth'].mean()
merged_df['well_depth']= merged_df['well_depth'].fillna(rep)

merged_df = merged_df.drop(columns = ['site_id', 'STATE', 'COUNTY', 'STATEFIPS', 'COUNTYFIPS'], axis = 1)
merged_df.to_csv('final_without_totals_visuals.csv', index = False)